# Implémentation

### 1. Installation des bibliothèques

In [2]:
!pip install -U langchain langchain-community langchain-core \
langchain-text-splitters sentence-transformers transformers pypdf accelerate bitsandbytes

!pip install chromadb==0.4.24

!pip install --force-reinstall protobuf==3.20.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.0/488.0 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 88.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 29.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.3/157.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━

### 2. Chargement des bibliothèques

In [3]:
import os
import torch

# LangChain
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline as hf_pipeline
from langchain_community.llms import HuggingFacePipeline

2025-11-28 15:45:10.614183: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764344710.840588      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764344710.904517      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### 3. CHARGEMENT DES PDF

In [4]:
def load_pdfs(pdf_paths):
    """
    Charge plusieurs fichiers PDF et renvoie une liste de Documents.
    Chaque page du PDF devient un Document (page_content + metadata).
    """
    all_docs = []
    for path in pdf_paths:
        if not os.path.exists(path):
            raise FileNotFoundError(f"Fichier introuvable : {path}")
        print(f" - Chargement du PDF : {path}")
        loader = PyPDFLoader(path)
        docs = loader.load()
        print(f"   {len(docs)} pages chargées")
        all_docs.extend(docs)

    print(f"\nTotal : {len(all_docs)} pages sur l'ensemble des PDF\n")
    return all_docs


### 4. DÉCOUPAGE EN CHUNKS

In [5]:
def split_documents(docs):
    """
    Découpe les Documents (pages PDF) en chunks plus petits.
    """
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,      # taille max d'un chunk en caractères
        chunk_overlap=200    # recouvrement entre chunks
    )
    chunks = splitter.split_documents(docs)
    print(f"{len(chunks)} chunks créés après découpage")
    return chunks

### 5. EMBEDDINGS + CHROMA

In [6]:
def build_vectorstore(chunks, persist_directory: str = "chroma_pdf_db"):
    """
    Crée une base vectorielle Chroma à partir des chunks.
    Utilise des embeddings SentenceTransformers.
    """
    print("Construction de la base vectorielle (embeddings Chroma)...")

    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )

    vectorstore = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory=persist_directory
    )

    print("Vectorstore prête\n")
    return vectorstore

### 6. PROMPT RAG TRÈS STRICT

In [7]:
RAG_PROMPT_TEMPLATE = """
Tu es un assistant RAG très STRICT.

Règles OBLIGATOIRES :
1. Tu utilises UNIQUEMENT le CONTEXTE ci-dessous (extraits des PDF).
2. Tu réponds STRICTEMENT dans la MÊME langue que la QUESTION
   (si la question est en français alors réponse en français,
   si elle est en anglais alors réponse en anglais, etc.).
3. Tu NE DOIS PAS utiliser de connaissances externes.
4. Si le contexte ne contient pas la réponse :
   - tu dis que tu ne sais pas,
   - tu précises que l'information n'est pas présente dans les PDF fournis,
   - toujours dans la même langue que la question.

CONTEXTE (extraits des PDF) :
{context}
QUESTION DE L'UTILISATEUR :
{question}

RÉPONSE (une seule langue, celle de la question) :
"""

def build_prompt():
    return PromptTemplate(
        input_variables=["context", "question"],
        template=RAG_PROMPT_TEMPLATE
    )

def format_docs(docs):
    """
    Concatène le contenu des Documents en une seule chaîne de contexte.
    """
    return "\n\n".join(doc.page_content for doc in docs)

### 7. LLM OPEN SOURCE FORT : Qwen2.5-7B-Instruct

In [9]:
def build_llm(model_name: str = "Qwen/Qwen2.5-7B-Instruct"):
    """
    Construit un LLM open source (Qwen2.5-7B-Instruct) via Transformers.
    Nécessite une machine GPU performante.
    """
    print(f"Chargement du modèle : {model_name}...")

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch.bfloat16
    )

    text_gen = hf_pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=256,
        do_sample=False,
        repetition_penalty=1.05
    )

    llm = HuggingFacePipeline(pipeline=text_gen)
    print("LLM Qwen2.5-7B prêt\n")
    return llm

### 8. CHAÎNE RAG (LCEL)

In [10]:
def build_rag_chain(vectorstore, llm):
    """
    Construit une chaîne RAG :
    question -> retriever -> formatage du contexte -> prompt -> LLM
    """
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 4}  # nombre de chunks récupérés
    )

    prompt = build_prompt()

    rag_chain = {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    } | prompt | llm

    return rag_chain


def answer_question(rag_chain, question: str) -> str:
    """
    Appelle la chaîne RAG sur une question donnée.
    """
    return rag_chain.invoke(question)

### 9. MAIN

In [ ]:
# MAIN
print("=== RAG LangChain + Qwen2.5-7B-Instruct + 2 PDF ===\n")

# 1) Charger les PDF
PDF_PATHS = ["/kaggle/input/tp-rag-dataset/Chapitre1.pdf", "/kaggle/input/tp-rag-dataset/Chapitre2.pdf"]
docs = load_pdfs(PDF_PATHS)

# 2) Découper en chunks
chunks = split_documents(docs)

# 3) Construire la base vectorielle
vectorstore = build_vectorstore(chunks)

# 4) LLM
llm = build_llm()

# 5) Chaîne RAG
rag_chain = build_rag_chain(vectorstore, llm)

print("Tes deux PDF sont indexés.")
print("Tape 'exit' pour quitter.\n")

while True:
    try:
        question = input(">>> Question : ").strip()
        if question.lower() in {"exit", "quit"}:
            print("Au revoir")
            break

        if not question:
            continue

        print("\n[Assistant] Réponse en cours...\n")
        try:
            response = answer_question(rag_chain, question)
            print(response)
            print("\n" + "-" * 60 + "\n")
        except Exception as e:
            print("Erreur lors de la génération de la réponse :", e)
            print("\n" + "-" * 60 + "\n")

    except (EOFError, KeyboardInterrupt):
        print("\nAu revoir")
        break

=== RAG LangChain + Qwen2.5-7B-Instruct + 2 PDF ===

 - Chargement du PDF : /kaggle/input/tp-rag-dataset/Chapitre1.pdf
   35 pages chargées
 - Chargement du PDF : /kaggle/input/tp-rag-dataset/Chapitre2.pdf
   44 pages chargées

Total : 79 pages sur l'ensemble des PDF

82 chunks créés après découpage
Construction de la base vectorielle (embeddings Chroma)...


/tmp/ipykernel_47/228020668.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...


[Assistant] Réponse en cours...


Tu es un assistant RAG très STRICT.

Règles OBLIGATOIRES :
1. Tu utilises UNIQUEMENT le CONTEXTE ci-dessous (extraits des PDF).
2. Tu réponds STRICTEMENT dans la MÊME langue que la QUESTION
   (si la question est en français alors réponse en français,
   si elle est en anglais alors réponse en anglais, etc.).
3. Tu NE DOIS PAS utiliser de connaissances externes.
4. Si le contexte ne contient pas la réponse :
   - tu dis que tu ne sais pas,
   - tu précises que l'information n'est pas présente dans les PDF fournis,
   - toujours dans la même langue que la question.

CONTEXTE (extraits des PDF) :
38
TransformersAttention
Créer de nouveaux embeddings en modifiant ceux qui existent 
déjà.

29
TransformersAttention
Et les autres mots ?

Chapitre 1 : 
Représentation des 
textes
01
02
03
04

35
TransformersAttention
Une seule représentation est-elle suffisante ?
No, Idéalement, nous aimerions avoir beaucoup d'intégration.
Problème : Construire de nombreux em

>>> Question :  dans quel slide il apparait "Step 4. Final Embedding" ?



[Assistant] Réponse en cours...


Tu es un assistant RAG très STRICT.

Règles OBLIGATOIRES :
1. Tu utilises UNIQUEMENT le CONTEXTE ci-dessous (extraits des PDF).
2. Tu réponds STRICTEMENT dans la MÊME langue que la QUESTION
   (si la question est en français alors réponse en français,
   si elle est en anglais alors réponse en anglais, etc.).
3. Tu NE DOIS PAS utiliser de connaissances externes.
4. Si le contexte ne contient pas la réponse :
   - tu dis que tu ne sais pas,
   - tu précises que l'information n'est pas présente dans les PDF fournis,
   - toujours dans la même langue que la question.

CONTEXTE (extraits des PDF) :
35
TransformersAttention
Une seule représentation est-elle suffisante ?
No, Idéalement, nous aimerions avoir beaucoup d'intégration.
Problème : Construire de nombreux embeddings en modifiant des embeddings existants.
Solution : Nous allons créer de nouveaux embeddings en modifiant ceux qui existent 
déjà.

Transformers
17
L’Embedding Layer avec Positional Encod

>>> Question :  le chapitre 2 explique quoi ?



[Assistant] Réponse en cours...


Tu es un assistant RAG très STRICT.

Règles OBLIGATOIRES :
1. Tu utilises UNIQUEMENT le CONTEXTE ci-dessous (extraits des PDF).
2. Tu réponds STRICTEMENT dans la MÊME langue que la QUESTION
   (si la question est en français alors réponse en français,
   si elle est en anglais alors réponse en anglais, etc.).
3. Tu NE DOIS PAS utiliser de connaissances externes.
4. Si le contexte ne contient pas la réponse :
   - tu dis que tu ne sais pas,
   - tu précises que l'information n'est pas présente dans les PDF fournis,
   - toujours dans la même langue que la question.

CONTEXTE (extraits des PDF) :
Chapitre 1 : 
Représentation des 
textes
01
02
03
04

Chapitre 2 : 
Transformers
02
03
04
05

Transformers
22
"The animal didn’t cross the street because it was too tired."
Transformers applique un mécanisme d’attention. 
Ce mécanisme d’attention 
permet à chaque token de 
la séquence de prêter 
attention à tous les autres 
tokens, ce qui capture les 
dépendanc